In [1]:
import requests
import mysql.connector
from mysql.connector import Error
import schedule
import time
 


In [2]:
# Airtable configuration
AIRTABLE_BASE_ID = 'appGnf8ldkBqyxsdX'
AIRTABLE_API_KEY = 'patb92KmkkX08CuWq.3e77882b6467bc2705d5f93a6468e7d74f8276494194126fe99328891d59f46b'
AIRTABLE_URL = f'https://api.airtable.com/v0/{AIRTABLE_BASE_ID}/'

# MySQL configuration
MYSQL_HOST = 'ugandarugbyunion.c5s2s2ki82x4.eu-north-1.rds.amazonaws.com'
MYSQL_DATABASE = 'ugandarugbyunion'
MYSQL_USER = 'admin'
MYSQL_PASSWORD = '2210kpeteR$$'

In [3]:
# Function to fetch records from Airtable
table_name = "facilities"
def fetch_airtable_records(table_name):
    headers = {
        'Authorization': f'Bearer {AIRTABLE_API_KEY}',
        'Content-Type': 'application/json'
    }
    response = requests.get(f'{AIRTABLE_URL}{table_name}', headers=headers)
    
    if response.status_code == 200:
        return response.json().get('records', [])
    else:
        print(f"Failed to fetch records from Airtable: {response.status_code}, Response: {response.text}")
        return []
records = fetch_airtable_records(table_name)
print(f"✅ {table_name}: {len(records)} records fetched.")

✅ facilities: 14 records fetched.


In [4]:
def upsert_record(cursor, table, record):
    fields = record.get('fields', {})

    if table == 'facilities':
        sql = """
        INSERT INTO facilities (Region, Facility, District)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            Region = VALUES(Region),
            District = VALUES(District);
        """

        values = (
            fields.get('Region'), 
            fields.get('Facility'),
            fields.get('District')
        )
        print(f"Executing SQL: {sql} with values: {values}")  # Logging

    else:
        print(f"⚠️ Table '{table}' not handled.")
        return

    try:
        cursor.execute(sql, values)
    except Error as err:
        print(f"Error executing SQL for table {table}: {err}")

In [5]:
def sync_airtable_to_mysql():
    try:
        connection = mysql.connector.connect(
            host=MYSQL_HOST,
            database=MYSQL_DATABASE,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD
        )
        
        if connection.is_connected():
            cursor = connection.cursor()

            # Fetch current records from MySQL for comparison
            cursor.execute("SELECT Facility FROM facilities;")
            existing_records = {row[0] for row in cursor.fetchall()}

            # Fetch records from Airtable
            records = fetch_airtable_records('facilities')  # Specify the table name
            
            # Prepare to track records that were found in Airtable
            fetched_facilities = set()

            for record in records:
                region = record['fields'].get('Region')
                facility = record['fields'].get('Facility')
                status = record['fields'].get('District')
                
                # Upsert record into MySQL
                upsert_record(cursor, 'facilities', record)

                # Track fetched referee names
                fetched_facilities.add(facility)

            # Deletions: Identify records in MySQL not found in Airtable
            for ref_name in existing_records:
                if ref_name not in fetched_facilities:
                    # Delete from MySQL if it's no longer in Airtable
                    delete_sql = "DELETE FROM referees WHERE RefName = %s;"
                    cursor.execute(delete_sql, (ref_name,))

            connection.commit()  # Commit all changes
            print("Data synchronized successfully, including deletions.")

    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if (connection.is_connected()):
            cursor.close()
            connection.close()
# Schedule the sync function to run every minute (adjust as needed)
schedule.every(1).minutes.do(sync_airtable_to_mysql)

# Keep the script running to maintain the scheduling
while True:
    schedule.run_pending()
    time.sleep(1)  # Sleep to prevent high CPU usage


Executing SQL: 
        INSERT INTO facilities (Region, Facility, District)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            Region = VALUES(Region),
            District = VALUES(District);
         with values: ('Central', 'Kyadondo Rugby Club', 'Kampala')
Executing SQL: 
        INSERT INTO facilities (Region, Facility, District)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            Region = VALUES(Region),
            District = VALUES(District);
         with values: ('Central', "King's Park Sports Arena Ltd", 'Wakiso')
Executing SQL: 
        INSERT INTO facilities (Region, Facility, District)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            Region = VALUES(Region),
            District = VALUES(District);
         with values: ('Central', 'Mandela National Stadium', 'Wakiso')
Executing SQL: 
        INSERT INTO facilities (Region, Facility, District)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE 
  

UnboundLocalError: cannot access local variable 'connection' where it is not associated with a value